In [6]:
import numpy as np
import json
import pandas as pd

In [7]:
import torch

In [8]:
data = json.load(open('data/intent.json','r'))

In [9]:
### No of intents do we have
f"No of intents : {len(data['intents'])}"

'No of intents : 22'

In [10]:
### what are those intent

", ".join([intent['intent'] for intent in data['intents']])

'Greeting, GreetingResponse, CourtesyGreeting, CourtesyGreetingResponse, CurrentHumanQuery, NameQuery, RealNameQuery, TimeQuery, Thanks, NotTalking2U, UnderstandQuery, Shutup, Swearing, GoodBye, CourtesyGoodBye, WhoAmI, Clever, Gossip, Jokes, PodBayDoor, PodBayDoorResponse, SelfAware'

In [11]:
# In each intent what are the things present
data['intents'][-1].keys()

dict_keys(['intent', 'text', 'responses', 'extension', 'context', 'entityType', 'entities'])

In [12]:
data['intents'][-3]

{'intent': 'PodBayDoor',
 'text': ['Open the pod bay door',
  'Can you open the pod bay door',
  'Will you open the pod bay door',
  'Open the pod bay door please',
  'Can you open the pod bay door please',
  'Will you open the pod bay door please',
  'Pod bay door'],
 'responses': ['Iâ€™m sorry, Iâ€™m afraid I canâ€™t do that!'],
 'extension': {'function': '', 'entities': False, 'responses': []},
 'context': {'in': '', 'out': 'PodBayDoor', 'clear': False},
 'entityType': 'NA',
 'entities': []}

In [13]:
intent_text = []
for intent in data['intents']:
    for text in intent['text']:
        intent_text.append({
            'text':text,
            'intent':intent['intent']
        })

In [14]:
intent_text_df  = pd.DataFrame(intent_text)

In [15]:
intent_text_df.head()

,text,intent
0,Hi,Greeting
1,Hi there,Greeting
2,Hola,Greeting
3,Hello,Greeting
4,Hello there,Greeting


In [16]:
intent_text_df.shape

(143, 2)

In [33]:
intent_text_df.to_csv("intent_text.csv",index=False)

In [17]:
intent_text_df.groupby(['intent']).count()

,text
intent,
Clever,7
CourtesyGoodBye,6
CourtesyGreeting,7
CourtesyGreetingResponse,8
CurrentHumanQuery,7
GoodBye,4
Gossip,6
Greeting,7
GreetingResponse,8


In [40]:
from transformers import AutoModel, BertTokenizerFast
# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# Import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
text = intent_text_df['text'].to_list()

In [75]:
t = tokenizer(text, padding=True,truncation=True, return_tensors='pt')

In [ ]:
tokenizer()

In [76]:
x , x_mask = t['input_ids'], t['attention_mask']

In [115]:
from sklearn.preprocessing import LabelEncoder

In [116]:
encoder = LabelEncoder()

In [117]:
y = encoder.fit_transform(intent_text_df['intent'].to_numpy().reshape(-1,1))

c:\users\deepa\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [168]:
encoder.inverse_transform([0])

array(['Clever'], dtype=object)

In [119]:
y.shape

(143,)

In [78]:
import torch.nn as nn

In [198]:
class BERT_Arch(torch.nn.Module):
    def __init__(self, bert=None):      
        super(BERT_Arch, self).__init__()
        self.max_len = 11
        self.bert = bert
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
        if not bert:
            self.bert = AutoModel.from_pretrained('bert-base-uncased')

            for p in self.bert.parameters():
                p.requires_grad = False
        else:
            self.bert = bert
        # dropout layer
        self.dropout = nn.Dropout(0.2)

        # relu activation function
        self.relu =  nn.ReLU()
        # dense layer
        self.fc1 = nn.Linear(768,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,22)
        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)
        #define the forward pass
    def forward(self, sent_id, mask):
        #pass the inputs to the model  
        cls_hs = self.bert(sent_id, attention_mask=mask)[0][:,0]

        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        # output layer
        x = self.fc3(x)

        # apply softmax activation
        x = self.softmax(x)
        return x
    
    def inference(self, text):
        label = ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting',
               'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye',
               'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery',
               'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse',
               'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks',
               'TimeQuery', 'UnderstandQuery', 'WhoAmI']
        with torch.no_grad():
            t= self.tokenizer(text, padding=True,truncation=True, return_tensors='pt',max_length=self.max_len)
            token, mask = t['input_ids'], t['attention_mask']
            res = self.forward(token, mask)
            return label[res.argmax()], torch.exp(res.max())

In [190]:
model = BERT_Arch()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [105]:
for param in bert.parameters():
    param.requires_grad = False

In [106]:
model = BERT_Arch(bert)

In [107]:
from transformers import AdamW
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

c:\users\deepa\appdata\local\programs\python\python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [108]:
from sklearn.utils.class_weight import compute_class_weight
#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(intent_text_df['intent']), intent_text_df['intent'])
print(class_wts)

[0.92857143 1.08333333 0.92857143 0.8125     0.92857143 1.625
 1.08333333 0.92857143 0.8125     1.08333333 1.08333333 0.92857143
 0.92857143 0.92857143 0.92857143 0.92857143 0.92857143 1.625
 1.08333333 0.92857143 1.08333333 1.08333333]


c:\users\deepa\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=['Clever' 'CourtesyGoodBye' 'CourtesyGreeting' 'CourtesyGreetingResponse'
 'CurrentHumanQuery' 'GoodBye' 'Gossip' 'Greeting' 'GreetingResponse'
 'Jokes' 'NameQuery' 'NotTalking2U' 'PodBayDoor' 'PodBayDoorResponse'
 'RealNameQuery' 'SelfAware' 'Shutup' 'Swearing' 'Thanks' 'TimeQuery'
 'UnderstandQuery' 'WhoAmI'], y=0       Greeting
1       Greeting
2       Greeting
3       Greeting
4       Greeting
         ...    
138    SelfAware
139    SelfAware
140    SelfAware
141    SelfAware
142    SelfAware
Name: intent, Length: 143, dtype: object as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [109]:
weights= torch.tensor(class_wts,dtype=torch.float)
# weights = weights.to(device)
# loss function
cross_entropy = nn.NLLLoss(weight=weights)

In [110]:
train_losses=[]
# number of training epochs
epochs = 200
# We can also use learning rate scheduler to achieve better results
lr_sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

In [120]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,x,m,y):
        self.x = x
        self.m = m
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.m[index], self.y[index]

dataset = Dataset(x,x_mask, y)
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=8,shuffle=True)

In [129]:
i[2].long()

tensor([15,  8, 15, 19,  3,  0, 11, 15])

In [121]:
for i in train_dataloader:
    print(i)
    break

[tensor([[  101,  2064,  2017,  6011,  2017,  2024,  2969,  1011,  5204,  3531,
           102],
        [  101,  2023,  2003, 12101,   102,     0,     0,     0,     0,     0,
             0],
        [  101,  2064,  2017,  6011,  2017,  2024,  2969,  5204,   102,     0,
             0],
        [  101,  2054,  2003,  1996,  2051,  1029,   102,     0,     0,     0,
             0],
        [  101,  2204,  4283,   999,  2023,  2003,  4205,   102,     0,     0,
             0],
        [  101, 12266,  2611,   102,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  2347,  1005,  1056,  4092,  2000,  2017,   102,     0,     0,
             0],
        [  101,  2064,  2017,  6011,  2017,  2031,  1037,  9715,   102,     0,
             0]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1,

In [130]:
def train():
  
    model.train()
    total_loss = 0  
    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):

        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step,    len(train_dataloader)))
        # push the batch to gpu
#         batch = [r.to(device) for r in batch] 
        sent_id, mask, labels = batch
        # get model predictions for the current batch
        preds = model(sent_id, mask)
        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels.long())
        # add on to the total loss
        total_loss = total_loss + loss.item()
        # backward pass to calculate the gradients
        loss.backward()
        # clip the the gradients to 1.0. It helps in preventing the    exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # update parameters
        optimizer.step()
        # clear calculated gradients
        optimizer.zero_grad()

        # We are not using learning rate scheduler as of now
        # lr_sch.step()
        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().numpy()
        # append the model predictions
        total_preds.append(preds)
        # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)
    #returns the loss and predictions
    return avg_loss, total_preds

In [131]:
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    # append training and validation loss
    train_losses.append(train_loss)
    # it can make your experiment reproducible, similar to set  random seed to all options where there needs a random seed.
print(f'\nTraining Loss: {train_loss:.3f}')


 Epoch 1 / 200

 Epoch 2 / 200

 Epoch 3 / 200

 Epoch 4 / 200

 Epoch 5 / 200

 Epoch 6 / 200

 Epoch 7 / 200

 Epoch 8 / 200

 Epoch 9 / 200

 Epoch 10 / 200

 Epoch 11 / 200

 Epoch 12 / 200

 Epoch 13 / 200

 Epoch 14 / 200

 Epoch 15 / 200

 Epoch 16 / 200

 Epoch 17 / 200

 Epoch 18 / 200

 Epoch 19 / 200

 Epoch 20 / 200

 Epoch 21 / 200

 Epoch 22 / 200

 Epoch 23 / 200

 Epoch 24 / 200

 Epoch 25 / 200

 Epoch 26 / 200

 Epoch 27 / 200

 Epoch 28 / 200

 Epoch 29 / 200

 Epoch 30 / 200

 Epoch 31 / 200

 Epoch 32 / 200

 Epoch 33 / 200

 Epoch 34 / 200

 Epoch 35 / 200

 Epoch 36 / 200

 Epoch 37 / 200

 Epoch 38 / 200

 Epoch 39 / 200

 Epoch 40 / 200

 Epoch 41 / 200

 Epoch 42 / 200

 Epoch 43 / 200

 Epoch 44 / 200

 Epoch 45 / 200

 Epoch 46 / 200

 Epoch 47 / 200

 Epoch 48 / 200

 Epoch 49 / 200

 Epoch 50 / 200

 Epoch 51 / 200

 Epoch 52 / 200

 Epoch 53 / 200

 Epoch 54 / 200

 Epoch 55 / 200

 Epoch 56 / 200

 Epoch 57 / 200

 Epoch 58 / 200

 Epoch 59 / 200

 Epoc

In [134]:
torch.save({
    'loss':train_loss,
    'param':model.state_dict()
},'cp/bert_intent.pt')

## Inference

In [199]:
model = BERT_Arch()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [200]:
cp = torch.load("cp/bert_intent.pt")
loss = cp['loss']
state_dict = cp['param']

In [201]:
model_stat = model.state_dict()
for p in model_stat:
    if p in state_dict:
        model_stat[p] = state_dict[p]
model.load_state_dict(model_stat)

<All keys matched successfully>

In [202]:
t,l = intent_text_df.sample(1).iloc[0].to_list()

In [203]:
l

'UnderstandQuery'

In [208]:
res, confidence = model.inference([t])

In [210]:
res, confidence

('UnderstandQuery', tensor(1.))